In [1]:
import numpy as np
import pandas as pd
from astropy.io import fits
from order_variables import order_variables
from nir_rv import NIR_RV
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# load telluric transmission spectrum
with fits.open('spec/atrans.fits') as hdul:
    atrans = hdul[0].data

In [3]:
# load standard 
wlcal = 1
atrest = True
stdrv = 18.2
file_tc ='spec/J0727+0513_rest.fits'
with fits.open(file_tc) as hdul:
    mystd = hdul[0].data
    shdr = hdul[0].header
mystd_tc = mystd

We'll test the code on a spectrum of the star TOI-1696 (data published here: https://ui.adsabs.harvard.edu/abs/2022AJ....163..298M/abstract).

In [4]:
# load non-telluric-corrected spectrum of target
file = "combspec_T470381900.fits"
with fits.open(file) as hdul:
    mydata = hdul[0].data
    hdr = hdul[0].header
    
# load telluric-corrected spectrum of target
file_tc = "tellcor_T470381900.fits"
with fits.open(file_tc) as hdul:
    mydata_tc = hdul[0].data
    
# load telluric-corrected, order-merged spectrum of target
file_merged = "merged_T470381900.fits"
with fits.open(file_merged) as hdul:
    merged_data = hdul[0].data

 ############################## Xspextool History ############################## [astropy.io.fits.card]
 ############################## Xcombspec History ############################## [astropy.io.fits.card]
 ############################### Xtellcor History ############################## [astropy.io.fits.card]
 ############################# Xmergeorders History ############################ [astropy.io.fits.card]


In [5]:
mydata_tc[:,0,:] = mydata[:,0,:]

In [6]:
# calculate the shift for the J, H, and K bands independently
RVs = np.zeros(3)
for order in range(3):
    # get pertinant variables for SpeX
    wrange, trange, pixscale, plorder = order_variables(hdr, order, instrument="spex")
    # calculate the RV
    myrv, torest, x2, mshft = NIR_RV(mydata_tc[order], hdr, mydata[order],
                              mystd_tc[order], shdr, mystd[order],
                              wlcal=wlcal, atrest=atrest, stdrv=stdrv, # already wavelength calibrated?
                              atrans=atrans,
                              pixscale=pixscale, plorder=plorder,
                              spixscale=pixscale, s_plorder=plorder,  # standard is from same set-up
                              wrange=wrange, trange=trange,
                              quiet=1)
    print("order =", order)
    print(myrv, torest, x2, mshft)
    if (order < 3):
        RVs[order] = myrv

/Users/stevengiacalone/Documents/research/mstars/tellrv/ern_rv.py:131: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  bin_idxs = np.array([np.argwhere((idxs > bin_edges[i]) & (idxs < bin_edges[i+1]))[:,0] for i in range(len(bin_edges)-1)])


order = 0
-5.4605598771922 -2.891476756292292 -2.891476756292292 -3.3854943213671247e-09
order = 1
0.009239982883129105 2.5783231037830374 2.5783231037830374 5.6044172748483426e-05
order = 2
0.11628309622483268 2.685366217124741 2.685366217124741 3.5837710012109994e-05


In [7]:
# take the median of the three 
this_RV = np.median(RVs) # our RV
this_RV

0.009239982883129105

In [8]:
# in this case, order 1 provides the values we need to shift the spectrum to the rest frame
torest, mshft = 2.5783231037830374, 5.6044172748483426e-05
shft = mshft - (mshft+merged_data[0])*torest/(3.e5)
shft

array([5.01150878e-05, 5.01140541e-05, 5.01130208e-05, ...,
       3.39803631e-05, 3.39772854e-05, 3.39742057e-05])

In [9]:
wav_shifted = merged_data[0] + shft
wav_shifted

array([0.68987094, 0.68999122, 0.69011144, ..., 2.56720579, 2.56756389,
       2.56792223])

In [10]:
# now save the data to a new file

# df = pd.DataFrame({'wavelength': wav_shifted, 'flux': merged_data[1], 'flux_err': merged_data[2]})
# df.to_csv('TOI1696_spex_restspectrum.csv', index=False)